# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure the API Key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('..\Output_Data\Weather_Data_Collection.csv')
weather_df = weather_df.drop(columns={'Unnamed: 0'})
weather_df.head()



,City,Lattitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,pindwara,24.80,73.07,30.38,47,0,2.68,IN,1601527781
1,linxia,35.60,103.21,14.58,55,86,1.62,CN,1601527781
2,kiblawan,6.61,125.24,32.00,59,40,5.10,PH,1601527782
3,atuona,-9.80,-139.03,25.79,77,3,9.78,PF,1601527783
4,kavieng,-2.57,150.80,29.15,70,93,2.15,PG,1601527784


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Obtaining the Coordinates to plot on the map
city_locations = weather_df[['Lattitude','Longitude']].astype(float)
city_locations.head()

,Lattitude,Longitude
0,24.80,73.07
1,35.60,103.21
2,6.61,125.24
3,-9.80,-139.03
4,-2.57,150.80


In [4]:
# Humidity values
humidity_data = weather_df['Humidity']
humidity_data.head()

0    47
1    55
2    59
3    77
4    70
Name: Humidity, dtype: int64

In [5]:
# Creating a Humidity Heatmap Layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(city_locations, weights=humidity_data, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(humidity_layer)

fig



Figure(layout=FigureLayout(height=&#39;420px&#39;))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# The most ideal vacations involve destinations with minimal cloud cover as well as having temperatures within the range of 20-30 degrees celcius. As such, there datframe will be narrowed down further in order to find hotels within these regions that have the ideal weather. 

ideal_weather_df = weather_df.loc[
    (weather_df['Cloudiness']==0) & 
    (weather_df['Wind Speed'] <= 10) & 
    (weather_df['Max Temp'] >= 20) &
    (weather_df['Max Temp'] <= 30)].dropna()

print(f' The Number of Ideal Cities within the ideal weather criteria are {len(ideal_weather_df)}')
ideal_weather_df


 The Number of Ideal Cities within the ideal weather criteria are 20


,City,Lattitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,caravelas,-17.71,-39.25,21.99,91,0,2.25,BR,1601527819
75,barkot,30.82,78.20,27.41,29,0,1.49,IN,1601527830
83,charlestown,-32.95,151.67,27.78,60,0,2.87,AU,1601527835
104,cuamba,-14.80,36.54,24.46,54,0,0.62,MZ,1601527847
110,aleppo,36.25,37.50,21.98,65,0,5.69,SY,1601527851
148,benghazi,32.12,20.07,21.64,68,0,1.74,LY,1601527876
201,bushehr,28.97,50.84,30.00,62,0,1.61,IR,1601527907
235,pahrump,36.21,-115.98,22.78,14,0,0.45,US,1601527836
240,kodinar,20.79,70.70,28.65,69,0,3.72,IN,1601527931
241,aden,12.78,45.04,29.84,70,0,5.42,YE,1601527931


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Creating a New Hotel DataFram
hotel_df = ideal_weather_df.copy().drop(columns={'Date', 'Wind Speed','Max Temp','Humidity','Cloudiness'})

hotel_df['Hotel Name'] = ""

hotel_df


,City,Lattitude,Longitude,Country,Hotel Name
60,caravelas,-17.71,-39.25,BR,
75,barkot,30.82,78.20,IN,
83,charlestown,-32.95,151.67,AU,
104,cuamba,-14.80,36.54,MZ,
110,aleppo,36.25,37.50,SY,
148,benghazi,32.12,20.07,LY,
201,bushehr,28.97,50.84,IR,
235,pahrump,36.21,-115.98,US,
240,kodinar,20.79,70.70,IN,
241,aden,12.78,45.04,YE,


In [8]:
# Setting up the parameters to determine the closest hotels within a 5 km radium (5000 m)

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Setting up the search parameters
params = {
    'type':'hotel',
    'keyword': 'hotel',
    'radius': 5000,
    'key': g_key
}

# We will be itterating through the rows in order to hotels within the defined radius of the coordinates of each city
print(f'Data retrival in progress')
for index, row in hotel_df.iterrows():
    latitude = row['Lattitude']
    longitude = row['Longitude']
    city = row['City']

    # Now that the latitude and longitude has been defined, these need to be added to the dictionary

    params['location'] = f'{latitude},{longitude}'

    
    response = requests.get(base_url, params=params).json()

    search_results = response['results']

    try:
        print(f'Finding nearest hotel for {city}\n--------------------------')
        hotel_df.loc[index, 'Hotel Name'] = search_results[0]['name']
    
    except (KeyError, IndexError):
        print(f'There are no Hotels for {city}, moving on to the next city....\n')


print('End of Data Retreival\n*********************')
print(f'Removing cities where hotels could not be retreived')
hotel_df = hotel_df[hotel_df['Hotel Name']!=""]




Data retrival in progress
Finding nearest hotel for caravelas
--------------------------
Finding nearest hotel for barkot
--------------------------
Finding nearest hotel for charlestown
--------------------------
Finding nearest hotel for cuamba
--------------------------
Finding nearest hotel for aleppo
--------------------------
There are no Hotels for aleppo, moving on to the next city....

Finding nearest hotel for benghazi
--------------------------
Finding nearest hotel for bushehr
--------------------------
Finding nearest hotel for pahrump
--------------------------
Finding nearest hotel for kodinar
--------------------------
Finding nearest hotel for aden
--------------------------
Finding nearest hotel for bam
--------------------------
Finding nearest hotel for sydney
--------------------------
Finding nearest hotel for carnarvon
--------------------------
Finding nearest hotel for mecca
--------------------------
Finding nearest hotel for jumla
--------------------------
F

In [9]:
# Results of the Hotel Search
hotel_df

,City,Lattitude,Longitude,Country,Hotel Name
60,caravelas,-17.71,-39.25,BR,Pousada Encanto Abrolhos
75,barkot,30.82,78.20,IN,HOTEL DIVINE (होटल डिवाइन)
83,charlestown,-32.95,151.67,AU,Pippi's At The Point Hotel
104,cuamba,-14.80,36.54,MZ,Daania Residence
148,benghazi,32.12,20.07,LY,Al-Fadheel Hotel
201,bushehr,28.97,50.84,IR,Delvar Hotel
235,pahrump,36.21,-115.98,US,Holiday Inn Express & Suites Pahrump
240,kodinar,20.79,70.70,IN,Vinnca Hotel and Banquets
241,aden,12.78,45.04,YE,Coral Hotel Aden
289,bam,29.11,58.36,IR,Parsian Bam Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lattitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
hotel_marker = gmaps.marker_layer(locations, info_box_content=hotel_info)

# We will add the new layer to the previous humidity heatmap
fig.add_layer(hotel_marker)

# Display figure
fig

Figure(layout=FigureLayout(height=&#39;420px&#39;))